In [2]:
!git add -A
!git commit -m "Add notebook to split dataset"
!git push origin master


*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@ea9c58b20389.(none)')
Username for 'https://github.com': ^C


In [1]:
%load_ext autoreload
%autoreload 2
import csv
import linecache
import string
import json
import os
from io import StringIO
import time

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchtext
from torchtext.data import get_tokenizer

from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Accuracy, Loss

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from models import SentimentClassifier
from datasets import *

In [2]:
max_tokens = 150
all_chars='abcdefghijklmnopqrstuvwxyz'
# all_chars='abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
batch_size = 256

In [3]:
dataset = LazyTextDataset(metadata_path="data/trustpilot_metadata.json",
                      max_tokens=max_tokens,
                      all_chars=all_chars,
                      strip_punctuations=False)


In [7]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=True, num_workers=4)
model = SentimentClassifier(max_tokens=max_tokens,
                         conv_num_kernels=[256]* 6,
                         conv_kernel_sizes=[5, 5, 3, 3, 3, 3],
                         pool_sizes=[None, None, None, None, None, 3])

In [8]:
model(next(iter(dataloader))[0]).shape

torch.Size([256, 3])

In [ ]:
device = 'cpu'
loss = F.cross_entropy
opt = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

trainer = create_supervised_trainer(model, opt, loss)
evaluator = create_supervised_evaluator(model,
                                        metrics={'accuracy': Accuracy(),
                                                 'cross_entropy': Loss(loss)},
                                        device=device)

log_interval = 1
desc = "Epoch {} - loss: {:.4f}"
pbar = tqdm(
    initial=0, leave=False, total=len(dataloader),
    desc=desc.format(1, 0)
)

@trainer.on(Events.ITERATION_COMPLETED)
def log_training_loss(engine):
    pbar.desc = desc.format(engine.state.epoch, engine.state.output)
    pbar.update(log_interval)

@trainer.on(Events.EPOCH_COMPLETED)
def log_training_results(engine):
    pbar.refresh()
    evaluator.run(dataloader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics['accuracy']
    avg_nll = metrics['cross_entropy']
    tqdm.write(
        "Training Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}"
        .format(engine.state.epoch, avg_accuracy, avg_nll)
    )

trainer.run(dataloader, max_epochs=100)